In [1]:
import pandas as pd
import dask.dataframe as dd
import glob as glob
import numpy as np
from scipy.stats import kde

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib.colors import LogNorm
plt.rc('font', size=16)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots

In [2]:
# input parameters for the simualtions
minE = 10**15 #eV
maxE = 10**18 #eV
gamma = -2.5 # slope of the spectrum

#relation between energy and s38
A = 10**12
B = 1.2

#attenuation true numbers
alpha = 0.5
beta = -3.0

# Minimum zenith angle in degrees
minTheta = 45

#number of events to generate
events = 10000
#number of bins for the zenith 
nr_of_bins = 8

import os
filepath='/home/noemie/stage/cic_att' 
os.chdir(filepath)
%run data_functions.py

from data_functions import generate_toy_data

data = generate_toy_data(events, minE, maxE, gamma, A, B, alpha, beta, minTheta)

In [3]:
###attenuation curve
from data_functions import obtain_attenuation

intensity=400
samples = 100

fit_results, fitted_data = obtain_attenuation(data, nr_of_bins, intensity, samples)

  0%|          | 0/100 [00:00<?, ?it/s]/home/noemie/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
100%|██████████| 100/100 [00:24<00:00,  4.12it/s]


emcee: Exception while calling your likelihood function:
  params: [  0.49356097  -2.88613903 482.76445395]
  args: (cos2
0.535717   -0.285677
0.607145   -0.214249
0.678573   -0.142821
0.750001   -0.071392
0.821430    0.000036
0.892858    0.071464
0.964286    0.142892
Name: cos2, dtype: float64, cos2
0.535717    301.234269
0.607145    366.565463
0.678573    418.241078
0.750001    456.261115
0.821430    480.625574
0.892858    491.334454
0.964286    488.387756
Name: cos2, dtype: float64, cos2
0.535717    2.043580
0.607145    2.741902
0.678573    2.619413
0.750001    3.300802
0.821430    3.616529
0.892858    3.479365
0.964286    3.146583
Name: s125, dtype: float64)
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/noemie/anaconda3/lib/python3.6/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/noemie/stage/cic_att/likelihood.py", line 67, in lnprob
    llh = lp + lnlike(params, cos2, y, yerr)
  File "/home/noemie/stage/cic_att/likelihood.py", line 21, in lnlike
    model = f * (b * cos2**2 + a * cos2 + 1)
  File "/home/noemie/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py", line 695, in wrapper
    def wrapper(left, right, name=name, na_op=na_op):
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import math
groups = fitted_data
sample = fit_results
cos_ref = np.cos(math.radians(25))**2
cos2 = np.linspace(0.5, 1, 20)-cos_ref

a_mcmc,  b_mcmc,  s38_mcmc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                  zip(*np.percentile(sample, [16, 50, 84],
                                      axis=0)))

#(mid_value, +error, -error)
print("a = %f + %f - %f\n"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
print("b = %f + %f - %f\n"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
print("s_ref = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))

# Plot a subset of the samples
plt.errorbar(groups.cos2.mean()-cos_ref, groups.s125.mean(), yerr= groups.s125.std().tolist(), fmt=".k")
for a, b, f in sample[np.random.randint(len(sample), size=250)]:
    plt.plot(cos2, f * (b * cos2**2 + a * cos2 + 1), color="b", alpha=0.03)
    plt.plot(cos2, s38_mcmc[0] * (b_mcmc[0] * cos2**2 + a_mcmc[0] * cos2 + 1), color="b", lw=0.5, alpha=0.8)
plt.errorbar(groups.cos2.mean()-cos_ref, groups.s125.mean(), yerr= groups.s125.std().tolist(), fmt=".k")

In [ ]:
import corner
fig = corner.corner(sample, truths=[alpha,beta,None],truth_color='r',labels=["$a$","$b$", "$s_{ref}$"], quantiles=[0.16, 0.84],
                    show_titles=True, title_kwargs={"fontsize": 20}, color='b')
#fig.savefig("")
a_mcmc,  b_mcmc,  s38_mcmc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(sample, [16, 50, 84],
                                                axis=0)))

#(mid_value, +error, -error)
print("a = %f + %f - %f\n"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
print("b = %f + %f - %f\n"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
print("s38 = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))

In [ ]:
fig2, ax2 =plt.subplots(figsize=(10,6))
temp, groups = set_intensity(data, 10)
for name, group in groups:
    ax2.plot(np.log10(group.s125), group.I, 
             label='$theta$ = {}'.format(math.floor(math.degrees(group.zenith.mean()))))
ax2.set_yscale("log", nonposy='clip')
plt.xlabel("$log_{10}(S_{125}/VEM)$")
plt.ylabel("Intensity  [a.u.]")

plt.legend()


In [ ]:
###plot a, b in fct of log(S_ref)

for a, b, f in sample[np.random.randint(len(sample), size=250)]:
    plt.plot(a,np.log10(f))

In [ ]:
%run data_functions.py
from data_functions import set_intensity
data['s38'] = data.s125/(b_mcmc[0] * (data.cos2-cos_ref)**2 + a_mcmc[0] * (data.cos2-cos_ref) + 1)

print(data)
temp, groups = set_intensity(data, 4, 's38')
for name, group in groups:
    ax2.plot(np.log10(group.s125), group.I, 
             label='$theta$ = {}'.format(math.floor(math.degrees(group.zenith.mean()))))
ax2.set_yscale("log", nonposy='clip')
plt.xlabel("$log_{10}(S_{125}/VEM)$")
plt.ylabel("Intensity  [a.u.]")

plt.legend()